In [1]:
import re, numpy as np, pandas as pd
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import numpy as np
import matplotlib.pyplot as plt

from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopWords_nltk = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /Users/lisun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('tripadvisor_hotel_reviews.csv')
df

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5
...,...,...
20486,"best kept secret 3rd time staying charm, not 5...",5
20487,great location price view hotel great quick pl...,4
20488,"ok just looks nice modern outside, desk staff ...",2
20489,hotel theft ruined vacation hotel opened sept ...,1


In [3]:
#will use only the first 1,000 rows
df_1000 = df[0:1000]

In [4]:
vectorizer = CountVectorizer(stop_words = 'english')
doc_term = vectorizer.fit_transform(df_1000.Review)

doc_term.shape

(1000, 8812)

In [9]:
tv = TfidfVectorizer(stop_words='english', max_df = 0.9, min_df = 5, ngram_range=(1,2))
tv_doc_term = tv.fit_transform(df.Review)
tv_doc_term2 = tv.transform(df.Review)
tfidf = pd.DataFrame(tv_doc_term.toarray(), columns = tv.get_feature_names())
tfidf

,00,00 00,00 10,00 11,00 12,00 15,00 30,00 allowed,00 best,00 bills,...,zoologischer,zoologischer garten,zurich,ä__ç_é_,äcor,äe,äes,äs,ù_,ùn
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20488,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20489,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
doc_term_df = pd.DataFrame(tv_doc_term.toarray(), columns=tv.get_feature_names())
doc_term_df

,00,00 00,00 10,00 11,00 12,00 15,00 30,00 allowed,00 best,00 bills,...,zoologischer,zoologischer garten,zurich,ä__ç_é_,äcor,äe,äes,äs,ù_,ùn
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20488,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20489,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
lsa = TruncatedSVD(4)
lsa.fit(tv_doc_term)
lsa

TruncatedSVD(n_components=4)

In [17]:
topic_term = lsa.components_.round(3)
topic_term

array([[ 0.024,  0.001,  0.   , ...,  0.001,  0.001,  0.   ],
       [ 0.029,  0.002,  0.   , ..., -0.   ,  0.001,  0.   ],
       [-0.032, -0.001, -0.   , ...,  0.   , -0.002,  0.   ],
       [-0.008, -0.   ,  0.   , ...,  0.   , -0.   ,  0.   ]])

In [18]:
topic_term_df = pd.DataFrame(topic_term.round(3),
                index = ["component_1", "component_2", "component_3", "component_4"], #"component_5"],
                columns = tv.get_feature_names())
topic_term_df

,00,00 00,00 10,00 11,00 12,00 15,00 30,00 allowed,00 best,00 bills,...,zoologischer,zoologischer garten,zurich,ä__ç_é_,äcor,äe,äes,äs,ù_,ùn
component_1,0.024,0.001,0.0,0.0,0.0,0.0,0.001,0.000,0.0,0.000,...,0.000,0.000,0.000,0.000,0.001,0.0,0.000,0.001,0.001,0.0
component_2,0.029,0.002,0.0,0.0,0.0,0.0,0.001,-0.000,-0.0,0.001,...,-0.000,-0.000,-0.000,-0.000,-0.001,-0.0,-0.001,-0.000,0.001,0.0
component_3,-0.032,-0.001,-0.0,0.0,-0.0,-0.0,-0.000,-0.001,-0.0,0.000,...,0.000,0.000,-0.000,0.000,-0.001,0.0,0.001,0.000,-0.002,0.0
component_4,-0.008,-0.000,0.0,0.0,0.0,0.0,0.000,-0.000,0.0,-0.000,...,0.002,0.002,0.001,0.001,0.000,-0.0,0.001,0.000,-0.000,0.0


In [19]:
# Function to display the top n terms in each topic
def display_topics(model, feature_names, no_top_words, topic_names = None): 
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix + 1)
        else:
            print("\nTopic: ", topic_names[ix])
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print("\n")
    return model, feature_names, no_top_words

In [21]:
output = display_topics(lsa, tv.get_feature_names(), 25)
### END SOLUTION
output;


Topic  1
hotel, room, great, good, stay, staff, nice, did, location, rooms, just, beach, service, night, breakfast, stayed, clean, time, resort, day, food, really, place, pool, like

Topic  2
resort, beach, food, pool, did, people, water, vacation, cana, punta, punta cana, time, beautiful, day, went, got, kids, like, fun, drinks, ocean, buffet, grounds, resorts, dominican

Topic  3
great, resort, beach, location, great location, excellent, friendly, staff, great hotel, helpful, hotel great, food, wonderful, recommend, loved, clean, beautiful, fantastic, perfect, best, staff friendly, restaurants, friendly helpful, highly, value

Topic  4
good, station, walk, value, breakfast, good value, nice, good location, train, metro, beach, clean, value money, bus, buffet, minutes, main, small, euros, hotel good, train station, minute walk, ok, area, 10




In [ ]:
def recommend(name, cosine_similarities = cosine_similarities):
    recommend_restaurant = []
    
    idx = indices[indices == name].index[0]
    
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
                             
                             
    top = list(score_series.iloc[0:50].index)
    
    for i in top:
        recommend_restaurant.append(list(df_percent.index)[i])
        
    df_new = pd.DataFrame(columns=['cuisines', "Mean Rating", 'cost'])
    
    for i in recommend_restaurant:
        df_new = df_new.append(pd.DataFrame(df_percent[['cuisines','Mean Rating', 'cost']][df_percent.index == i].sample()))
        
    df_new = df_new.drop_duplicates(subset=['cuisines', 'Mean Rating', 'cost'], keep=False)
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)

    print('Top 10 Restaurants with Similar Reviews:')
    
    return df_new